In [2]:
import requests as rq
import json
import pandas as pd

In [136]:
class scb:
    def __init__(self, language='sv', levels=None, query=None):
        self.language = language
        self.url = 'http://api.scb.se/OV0104/v1/doris/{}/ssd/'.format(self.language)
        self.levels = None
        self.data = None
        self.dataframe = None
        self.variables = None
        self.table_name = None
        self.get_data(self.url, levels, query)
        
    def get_data(self, url, levels, query):
        if levels:
            print('Levels:')
            self.levels = levels
            for l in levels:
                print(l)
                url += l + '/'
            self.url = url
        with rq.get(url) as response:
            self.response = response
            self.data = response.json()
            if type(self.data) is list:
                #self.dataframe = pd.read_json(json.dumps(response.json()), orient='records')
                self.variables = None
                self.table_name = None
                self.dataframe = pd.read_json(response.content, orient='records')
            elif type(self.data) is dict:
                self.dataframe = None
                self.variables = self.data['variables']
                self.table_name = self.data['title']
                if query:
                    self.query = {'query': [], 
                                  'response': {'format': 'json'}}
                    for q in query:
                        self.query['query'].append({'code': q['code'],
                                                    'selection': {'filter': 
                                                                  'item', 
                                                                  'values': q['values']}})
                    with rq.post(url, json=self.query) as response:
                        self.response = response
                        self.data = response.json()
                        index = []
                        columns = []
                        values = []
                        
                        for k in scbr.data['columns']:
                            if k['type'] == 'c':
                                columns.append(k['text'])
                                
                                
                        for k in scbr.data['data']:
                            index.append(k['key'])
                            values.append(k['values'])
                            
                        self.dataframe = pd.read_json(
                            json.dumps(
                                {'index': index, 
                                 'columns':columns, 
                                 'data':values}), orient='split')
                    
            else:
                self.dataframe = None
                
#     def set_query(self, )
        
            
    def __repr__(self):
        return str(self.data)

In [137]:
query = [{'code':'SNI2007', 'values':['J', 'Q']}, 
         {'code': 'Storleksklass', 'values': ['SGR0',
                                              'SGR1',
                                              'SGR2',
                                              'SGR3',
                                              'SGR4',
                                              'SGR5',
                                              'SGR6',
                                              'SGR7',
                                              'SGR8']}, 
         {'code': 'ContentsCode', 'values': ['000002YB']},
         {'code': 'Tid', 'values': ['2018', '2019']}]

scbr = scb(levels=['NV', 'NV0101', 'FDBR07N'], query=query)


Levels:
NV
NV0101
FDBR07N
Has query: [{'code': 'SNI2007', 'values': ['J', 'Q']}, {'code': 'Storleksklass', 'values': ['SGR0', 'SGR1', 'SGR2', 'SGR3', 'SGR4', 'SGR5', 'SGR6', 'SGR7', 'SGR8']}, {'code': 'ContentsCode', 'values': ['000002YB']}, {'code': 'Tid', 'values': ['2018', '2019']}]


ValueError: Shape of passed values is (36, 1), indices imply (3, 1)

In [135]:
scbr.dataframe

,Företag (FDB)
2018,41354
2019,41792
2018,17841
2019,19272
2018,1768
2019,1829
2018,1180
2019,1228
2018,794
2019,856


In [103]:
# scbpd = pd.read_json(json.dumps({'index': [for k in scbr.data['columns'][0]]}), )

index = []
columns = []
values = []

print(scbr.data['columns'])

for k in scbr.data['columns']:
    if k['type'] == 'c':
        columns.append(k['text'])
        
for k in scbr.data['data']:
    index.append(k['key'][-1])
    values.append(k['values'])
    
    
js = pd.read_json(json.dumps({'index': index, 'columns':columns, 'data':values}), orient='split')
js

[{'code': 'NyckeltalSCB', 'text': 'typ av nyckeltal', 'type': 'd'}, {'code': 'Tid', 'text': 'månad', 'type': 't'}, {'code': '000001DE', 'text': 'Nyckeltal', 'type': 'c'}, {'code': '000001DD', 'text': 'Förändring mot föregående månad, procent', 'type': 'c'}, {'code': '000001D9', 'text': 'Förändring mot samma månad föregående år, procent', 'type': 'c'}]


,Nyckeltal,"Förändring mot föregående månad, procent","Förändring mot samma månad föregående år, procent"
2020M06,42620,-0.1,1.7
